In [1]:
import astropy
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table

In [2]:
#Import Initial CSV Files of APOGEE (df) and the Yu Radii (df2) and combine
df = pd.read_csv(r'Targets+2_12.csv')
df2 = pd.read_csv(r'radii.csv')

df3 = df.merge(df2, how='outer',left_on="SDSS ID", right_on="starID")
dfyuapo = df3.query("nvisits>1")

df_fircomb = dfyuapo.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
df_fircomb.query('radius>0', inplace=True)
df_fircomb = df_fircomb.rename(columns={"radius": "Radius [Solar Radii]"})
df_fircomb


,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,Effective Temperature Error,logg,Log(g) Error,vscatter,vsini,SLs,FE/H,FE/H Error,starID,Radius [Solar Radii]
0,4287.0,120+12,apo25m,2M00000662+7528598,5.396841e+17,9.0,773.235000,9.435,0.027,9.268,...,66.630350,3.883139,0.023815,2.006593,37.033950,1.0,NaN,NaN,2M00000662+7528598,2.096
1,5218.0,100-60,apo25m,2M00005343+0040594,2.738249e+18,4.0,95.417015,11.193,0.023,10.910,...,49.525173,4.453109,0.034854,4.180007,20.137243,1.0,-0.136960,-0.136960,2M00005343+0040594,1.574
2,4548.0,105-45,apo25m,2M00020972+1612294,2.772432e+18,3.0,548.875730,9.211,0.019,9.077,...,75.828450,3.786097,0.029353,10.923258,53.112904,1.0,NaN,NaN,2M00020972+1612294,2.190
3,2897.0,107-46_MGA,apo25m,2M00020972+1612294,2.772432e+18,2.0,196.888920,9.211,0.019,9.077,...,64.904310,3.734142,0.029499,14.031831,56.221184,1.0,-0.274090,-0.274090,2M00020972+1612294,2.190
4,5218.0,100-60,apo25m,2M00021917+0142107,2.738655e+18,4.0,243.988720,10.456,0.023,9.996,...,14.205135,4.449954,0.020621,1.507434,11.972918,1.0,-0.171120,-0.171120,2M00021917+0142107,1.392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5385,4264.0,N7789,apo25m,2M23582919+5601109,1.994739e+18,3.0,77.150215,12.300,0.023,12.021,...,61.388924,4.159441,0.054019,2.986133,12.557406,1.0,-0.018182,-0.018182,2M23582919+5601109,1.198
5386,4287.0,120+12,apo25m,2M23591060+7448266,2.229207e+18,9.0,1993.943200,7.273,0.018,7.212,...,55.047024,3.905743,0.023300,13.648273,66.160690,1.0,0.051643,0.051643,2M23591060+7448266,2.449
5387,4548.0,105-45,apo25m,2M23592268+1714293,2.773674e+18,3.0,310.058870,9.839,0.022,9.620,...,55.675426,3.856124,0.029318,3.040208,31.521002,1.0,-0.235250,-0.235250,2M23592268+1714293,3.096
5388,7218.0,SMC12,lco25m,2M23594797-7254435,6.380061e+18,12.0,896.137450,9.818,0.024,9.682,...,45.777060,3.914288,0.020989,12.275522,86.317730,1.0,0.097861,0.097861,2M23594797-7254435,2.173


In [3]:
#Now we import the Joker data and slim it to the necessary Data

data = Table.read('apJoker-metadata.fits', format='fits')
jokerdf = data.to_pandas()


import re

def extract_id(value):
    return re.search(r"b'(.+)'", str(value)).group(1)
jokerdf['APOGEE_ID'] = jokerdf['APOGEE_ID'].apply(extract_id)

slimmedjokerdf = jokerdf[['APOGEE_ID', 'MAP_P', 'MAP_e', 'MAP_K']].copy()
slimmedjokerdf = slimmedjokerdf.rename(columns={"MAP_P": "Period [Joker, Days]",
                                   "MAP_e": "Eccentricity [Joker]", "MAP_K": "K [Joker, km/s]"})

changedperiod = slimmedjokerdf["Period [Joker, Days]"].mul(86400)
df10 = pd.DataFrame(changedperiod)


slimmedjokerdf = slimmedjokerdf.rename(columns={"Period [Joker, Days]": "Period [Joker, Seconds]"})
slimmedjokerdf["Period [Joker, Seconds]"] = 0
slimmedjokerdf["Period [Joker, Seconds]"] = df10
slimmedjokerdf

,APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]"
0,2M00000002+7417074,3.145566e+06,0.419240,16.410231
1,2M00000019-1924498,3.612605e+05,0.000712,33.166533
2,2M00000032+5737103,3.145566e+06,0.419240,16.410231
3,2M00000068+5710233,1.381036e+05,0.659999,47.022936
4,2M00000133+5721163,3.145566e+06,0.419240,16.410231
...,...,...,...,...
358345,AP18341149-2302059,1.984497e+05,0.767292,0.139080
358346,FF_Aql,1.297341e+05,0.110467,23.033727
358347,FN_Aql,1.296254e+05,0.102757,85.302459
358348,GQ_Ori,1.321291e+05,0.089768,23.796041


In [4]:
df3 = df_fircomb.merge(slimmedjokerdf, how='outer',left_on="SDSS ID", right_on="APOGEE_ID")
df4 = df3.query("nvisits>1")
df_wjoker = df4

df_wjoker #this returns 5930 objects

#df_wjoker.query("`Radius [Solar Radii]` > 0", inplace=True)
#df_wjoker #cutting the ones without Yu et al. radii, we get down to 4764 objects


,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,vsini,SLs,FE/H,FE/H Error,starID,Radius [Solar Radii],APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]"
0,4287.0,120+12,apo25m,2M00000662+7528598,5.396841e+17,9.0,773.235000,9.435,0.027,9.268,...,37.033950,1.0,NaN,NaN,2M00000662+7528598,2.096,NaN,NaN,NaN,NaN
1,5218.0,100-60,apo25m,2M00005343+0040594,2.738249e+18,4.0,95.417015,11.193,0.023,10.910,...,20.137243,1.0,-0.136960,-0.136960,2M00005343+0040594,1.574,2M00005343+0040594,198580.084991,0.234826,54.428057
2,4548.0,105-45,apo25m,2M00020972+1612294,2.772432e+18,3.0,548.875730,9.211,0.019,9.077,...,53.112904,1.0,NaN,NaN,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN
3,2897.0,107-46_MGA,apo25m,2M00020972+1612294,2.772432e+18,2.0,196.888920,9.211,0.019,9.077,...,56.221184,1.0,-0.274090,-0.274090,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN
4,5218.0,100-60,apo25m,2M00021917+0142107,2.738655e+18,4.0,243.988720,10.456,0.023,9.996,...,11.972918,1.0,-0.171120,-0.171120,2M00021917+0142107,1.392,2M00021917+0142107,132221.712112,0.239191,3.508314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,4264.0,N7789,apo25m,2M23582919+5601109,1.994739e+18,3.0,77.150215,12.300,0.023,12.021,...,12.557406,1.0,-0.018182,-0.018182,2M23582919+5601109,1.198,2M23582919+5601109,176730.503082,0.406749,97.596803
4760,4287.0,120+12,apo25m,2M23591060+7448266,2.229207e+18,9.0,1993.943200,7.273,0.018,7.212,...,66.160690,1.0,0.051643,0.051643,2M23591060+7448266,2.449,NaN,NaN,NaN,NaN
4761,4548.0,105-45,apo25m,2M23592268+1714293,2.773674e+18,3.0,310.058870,9.839,0.022,9.620,...,31.521002,1.0,-0.235250,-0.235250,2M23592268+1714293,3.096,NaN,NaN,NaN,NaN
4762,7218.0,SMC12,lco25m,2M23594797-7254435,6.380061e+18,12.0,896.137450,9.818,0.024,9.682,...,86.317730,1.0,0.097861,0.097861,2M23594797-7254435,2.173,NaN,NaN,NaN,NaN


In [5]:
#Calculates Rotational Periods in seconds in case the Period from joker doesn't exist

for i in range (len(df_wjoker["Gaia ID"])):
    vsini = df_wjoker['vsini'][i]
    Radius_km = (df_wjoker["Radius [Solar Radii]"][i])* 695700
    pi = np.pi
    per = (((2 * pi) * Radius_km) / vsini)
    #print(per)
    df_wjoker.loc[i,'Period [seconds, Rotational]'] = per

df_wjoker

/var/folders/s6/5bg03ycx39q47jb7qlhrs1s80000gn/T/ipykernel_94831/3376933635.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wjoker.loc[i,'Period [seconds, Rotational]'] = per


,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,SLs,FE/H,FE/H Error,starID,Radius [Solar Radii],APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]","Period [seconds, Rotational]"
0,4287.0,120+12,apo25m,2M00000662+7528598,5.396841e+17,9.0,773.235000,9.435,0.027,9.268,...,1.0,NaN,NaN,2M00000662+7528598,2.096,NaN,NaN,NaN,NaN,247396.251012
1,5218.0,100-60,apo25m,2M00005343+0040594,2.738249e+18,4.0,95.417015,11.193,0.023,10.910,...,1.0,-0.136960,-0.136960,2M00005343+0040594,1.574,2M00005343+0040594,198580.084991,0.234826,54.428057,341669.796439
2,4548.0,105-45,apo25m,2M00020972+1612294,2.772432e+18,3.0,548.875730,9.211,0.019,9.077,...,1.0,NaN,NaN,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN,180237.825442
3,2897.0,107-46_MGA,apo25m,2M00020972+1612294,2.772432e+18,2.0,196.888920,9.211,0.019,9.077,...,1.0,-0.274090,-0.274090,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN,170273.082827
4,5218.0,100-60,apo25m,2M00021917+0142107,2.738655e+18,4.0,243.988720,10.456,0.023,9.996,...,1.0,-0.171120,-0.171120,2M00021917+0142107,1.392,2M00021917+0142107,132221.712112,0.239191,3.508314,508207.533814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,4264.0,N7789,apo25m,2M23582919+5601109,1.994739e+18,3.0,77.150215,12.300,0.023,12.021,...,1.0,-0.018182,-0.018182,2M23582919+5601109,1.198,2M23582919+5601109,176730.503082,0.406749,97.596803,417021.795569
4760,4287.0,120+12,apo25m,2M23591060+7448266,2.229207e+18,9.0,1993.943200,7.273,0.018,7.212,...,1.0,0.051643,0.051643,2M23591060+7448266,2.449,NaN,NaN,NaN,NaN,161804.513112
4761,4548.0,105-45,apo25m,2M23592268+1714293,2.773674e+18,3.0,310.058870,9.839,0.022,9.620,...,1.0,-0.235250,-0.235250,2M23592268+1714293,3.096,NaN,NaN,NaN,NaN,429341.440617
4762,7218.0,SMC12,lco25m,2M23594797-7254435,6.380061e+18,12.0,896.137450,9.818,0.024,9.682,...,1.0,0.097861,0.097861,2M23594797-7254435,2.173,NaN,NaN,NaN,NaN,110042.788609


In [6]:
#Calculates Beta using Joker Radius 
for i in range (len(df_wjoker["SDSS ID"])):
    K = df_wjoker["K [Joker, km/s]"][i]
    period = df_wjoker["Period [Joker, Seconds]"][i]
    e = df_wjoker["Eccentricity [Joker]"][i]
    grav = 6.6743 * (10**-20)
    pi = np.pi
    df_wjoker.loc[i, "Beta [Solar Masses, Joker]"] = ((((K**3) * period) / (2 * pi * grav)) * 5.02785e-31 * ((1 - e)**(3/2)))
df_wjoker


#Calculates Beta using Derived Rotational Period Radius 
for i in range (len(df_wjoker["SDSS ID"])):
    vscatter = df_wjoker["vscatter"][i]
    period = df_wjoker["Period [seconds, Rotational]"][i]
    grav = 6.6743 * (10**-20)
    pi = np.pi
    df_wjoker.loc[i, "Beta [Solar Masses, Rotational Period]"] = ((((vscatter**3) * period) / (2 * pi * grav)) * 5.02785e-31)
df_wjoker

/var/folders/s6/5bg03ycx39q47jb7qlhrs1s80000gn/T/ipykernel_94831/4236222113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wjoker.loc[i, "Beta [Solar Masses, Joker]"] = ((((K**3) * period) / (2 * pi * grav)) * 5.02785e-31 * ((1 - e)**(3/2)))
/var/folders/s6/5bg03ycx39q47jb7qlhrs1s80000gn/T/ipykernel_94831/4236222113.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wjoker.loc[i, "Beta [Solar Masses, Rotational Period]"] = ((((vscatter**3) * period) / (2 * pi * grav)) * 5.02785e-31)


,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,FE/H Error,starID,Radius [Solar Radii],APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]","Period [seconds, Rotational]","Beta [Solar Masses, Joker]","Beta [Solar Masses, Rotational Period]"
0,4287.0,120+12,apo25m,2M00000662+7528598,5.396841e+17,9.0,773.235000,9.435,0.027,9.268,...,NaN,2M00000662+7528598,2.096,NaN,NaN,NaN,NaN,247396.251012,NaN,0.000002
1,5218.0,100-60,apo25m,2M00005343+0040594,2.738249e+18,4.0,95.417015,11.193,0.023,10.910,...,-0.136960,2M00005343+0040594,1.574,2M00005343+0040594,198580.084991,0.234826,54.428057,341669.796439,0.025695,0.000030
2,4548.0,105-45,apo25m,2M00020972+1612294,2.772432e+18,3.0,548.875730,9.211,0.019,9.077,...,NaN,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN,180237.825442,NaN,0.000282
3,2897.0,107-46_MGA,apo25m,2M00020972+1612294,2.772432e+18,2.0,196.888920,9.211,0.019,9.077,...,-0.274090,2M00020972+1612294,2.190,NaN,NaN,NaN,NaN,170273.082827,NaN,0.000564
4,5218.0,100-60,apo25m,2M00021917+0142107,2.738655e+18,4.0,243.988720,10.456,0.023,9.996,...,-0.171120,2M00021917+0142107,1.392,2M00021917+0142107,132221.712112,0.239191,3.508314,508207.533814,0.000005,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,4264.0,N7789,apo25m,2M23582919+5601109,1.994739e+18,3.0,77.150215,12.300,0.023,12.021,...,-0.018182,2M23582919+5601109,1.198,2M23582919+5601109,176730.503082,0.406749,97.596803,417021.795569,0.090006,0.000013
4760,4287.0,120+12,apo25m,2M23591060+7448266,2.229207e+18,9.0,1993.943200,7.273,0.018,7.212,...,0.051643,2M23591060+7448266,2.449,NaN,NaN,NaN,NaN,161804.513112,NaN,0.000493
4761,4548.0,105-45,apo25m,2M23592268+1714293,2.773674e+18,3.0,310.058870,9.839,0.022,9.620,...,-0.235250,2M23592268+1714293,3.096,NaN,NaN,NaN,NaN,429341.440617,NaN,0.000014
4762,7218.0,SMC12,lco25m,2M23594797-7254435,6.380061e+18,12.0,896.137450,9.818,0.024,9.682,...,0.097861,2M23594797-7254435,2.173,NaN,NaN,NaN,NaN,110042.788609,NaN,0.000244


In [7]:
#SM = star mass; Beta = Constants [solar mass]; CO =compact object mass

#First lets get CO masses using the Joker parameters
for i in range (len(df_wjoker["SDSS ID"])):
    beta = df_wjoker["Beta [Solar Masses, Joker]"][i]
    b_3 = float(beta**3)
    b_2 = float(beta**2)
    SM_4 = float(1)
    SM_3 = float(1)
    SM_2 = float(1)
    SM_1 = float(1)
    term_1 = (.26457*(((27.0 * SM_2 * beta) + (5.1962 * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3)))
    term_2_num = .41997*(-1*((6.0 * SM_1 * beta) + b_2))
    #print(term_2_num)
    term_2_dem = (((27.0 * SM_2 * beta) + (5.1962 * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3))
    #print(term_2_dem)
    term_2 = term_2_num / term_2_dem
    term_3 = (.33333) * beta
    df_wjoker.loc[i, "CO Mass [Solar Masses, Joker]"] = (term_1 - term_2 + term_3 + 1.0)



#Now from Rotational Period Beta

for i in range (len(df_wjoker["SDSS ID"])):
    beta = df_wjoker["Beta [Solar Masses, Rotational Period]"][i]
    b_3 = float(beta**3)
    b_2 = float(beta**2)
    SM_4 = float(1)
    SM_3 = float(1)
    SM_2 = float(1)
    SM_1 = float(1)
    term_1 = (.26457*(((27.0 * SM_2 * beta) + (5.1962 * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3)))
    term_2_num = .41997*(-1*((6.0 * SM_1 * beta) + b_2))
    #print(term_2_num)
    term_2_dem = (((27.0 * SM_2 * beta) + (5.1962 * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3))
    #print(term_2_dem)
    term_2 = term_2_num / term_2_dem
    term_3 = (.33333) * beta
    df_wjoker.loc[i, "CO Mass [Solar Masses, Rotational Period]"] = (term_1 - term_2 + term_3 + 1.0)
df_wjoker

/var/folders/s6/5bg03ycx39q47jb7qlhrs1s80000gn/T/ipykernel_94831/714099258.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wjoker.loc[i, "CO Mass [Solar Masses, Joker]"] = (term_1 - term_2 + term_3 + 1.0)
/var/folders/s6/5bg03ycx39q47jb7qlhrs1s80000gn/T/ipykernel_94831/714099258.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wjoker.loc[i, "CO Mass [Solar Masses, Rotational Period]"] = (term_1 - term_2 + term_3 + 1.0)


,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,Radius [Solar Radii],APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]","Period [seconds, Rotational]","Beta [Solar Masses, Joker]","Beta [Solar Masses, Rotational Period]","CO Mass [Solar Masses, Joker]","CO Mass [Solar Masses, Rotational Period]"
0,4287.0,120+12,apo25m,2M00000662+7528598,5.396841e+17,9.0,773.235000,9.435,0.027,9.268,...,2.096,NaN,NaN,NaN,NaN,247396.251012,NaN,0.000002,NaN,1.013502
1,5218.0,100-60,apo25m,2M00005343+0040594,2.738249e+18,4.0,95.417015,11.193,0.023,10.910,...,1.574,2M00005343+0040594,198580.084991,0.234826,54.428057,341669.796439,0.025695,0.000030,1.362703,1.031697
2,4548.0,105-45,apo25m,2M00020972+1612294,2.772432e+18,3.0,548.875730,9.211,0.019,9.077,...,2.190,NaN,NaN,NaN,NaN,180237.825442,NaN,0.000282,NaN,1.068511
3,2897.0,107-46_MGA,apo25m,2M00020972+1612294,2.772432e+18,2.0,196.888920,9.211,0.019,9.077,...,2.190,NaN,NaN,NaN,NaN,170273.082827,NaN,0.000564,NaN,1.087368
4,5218.0,100-60,apo25m,2M00021917+0142107,2.738655e+18,4.0,243.988720,10.456,0.023,9.996,...,1.392,2M00021917+0142107,132221.712112,0.239191,3.508314,508207.533814,0.000005,0.000002,1.016746,1.012889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,4264.0,N7789,apo25m,2M23582919+5601109,1.994739e+18,3.0,77.150215,12.300,0.023,12.021,...,1.198,2M23582919+5601109,176730.503082,0.406749,97.596803,417021.795569,0.090006,0.000013,1.617536,1.024080
4760,4287.0,120+12,apo25m,2M23591060+7448266,2.229207e+18,9.0,1993.943200,7.273,0.018,7.212,...,2.449,NaN,NaN,NaN,NaN,161804.513112,NaN,0.000493,NaN,1.083340
4761,4548.0,105-45,apo25m,2M23592268+1714293,2.773674e+18,3.0,310.058870,9.839,0.022,9.620,...,3.096,NaN,NaN,NaN,NaN,429341.440617,NaN,0.000014,NaN,1.024766
4762,7218.0,SMC12,lco25m,2M23594797-7254435,6.380061e+18,12.0,896.137450,9.818,0.024,9.682,...,2.173,NaN,NaN,NaN,NaN,110042.788609,NaN,0.000244,NaN,1.065180


In [17]:
for i in range (len(df_wjoker["SDSS ID"])):
    estimated = df_wjoker["CO Mass [Solar Masses, Rotational Period]"][i]
    true = df_wjoker["CO Mass [Solar Masses, Joker]"][i]
    diff = estimated - true
    final = '%.3f'%(abs(diff / true) * 100)
    df_wjoker.loc[i, "Percent Error Mass"] = str(final)
df_wjoker


greenobjs_joker = df_wjoker.query('`CO Mass [Solar Masses, Joker]` > 2.17 & `Period [Joker, Seconds]` > 1')
greenobjs_joker

#There are almost 607 with CO Mass > 1.4!! only about 127 with Mass > 2.17. Adding a top mass
#restriction of 5, we have 601 > 1.4M, and 121 Mass > 2.17

#Star #126 is the WORST one I have ever seen lmao
#Star 5369 is amazing

#Star #17 is only in the rotational period (CO mass is ~1.5) but error is really low
#star #4094 is also rlly good only in RP ones (CO mass ~1.42-43)

greenobjs_rp = df_wjoker.query('`CO Mass [Solar Masses, Rotational Period]` > 1.4')

In [18]:
greenobjs_joker.shape

(2, 40)

In [13]:
#Use this to check if an object in the Rotational Period Array is in the Joker Array too

print(len(greenobjs_joker.count(axis='rows')))
len1 = len(greenobjs_joker['Field'])
len2 = len(greenobjs_joker.count(axis='rows'))
for i in range(len1):
    for j in range (len2):
        if greenobjs_joker.iat[i, j] == 5906.0: #change this to the "Location ID" you want to verify
            print("YES!")
            break

40
YES!


In [10]:
printer = greenobjs_joker.sort_values(by="Percent Error Mass")
printer

,Location ID,Field,Telescope,SDSS ID,Gaia ID,nvisits,Signal-to-noise,J-Band Magnitude,J-Band Error,H-Band Magnitude,...,APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]","Period [seconds, Rotational]","Beta [Solar Masses, Joker]","Beta [Solar Masses, Rotational Period]","CO Mass [Solar Masses, Joker]","CO Mass [Solar Masses, Rotational Period]",Percent Error Mass
2613,2342.0,152+63_MGA,apo25m,2M11291785+4853250,7.891989e+17,3.0,166.062130,10.015,0.030,9.704,...,2M11291785+4853250,1.780840e+05,0.171288,130.673955,113131.573475,0.359413,0.077525,2.204205,1.577928,28.413
2763,5297.0,M68,lco25m,2M12395622-2630087,3.496402e+18,3.0,113.151924,12.482,0.024,12.082,...,2M12395622-2630087,1.795383e+05,0.001101,127.283267,551121.724091,0.443150,0.092320,2.346125,1.624594,30.754
335,5599.0,146-24_btx,apo25m,2M02270842+3423205,3.267540e+17,3.0,491.644260,9.717,0.023,9.145,...,2M02270842+3423205,1.924322e+05,0.013608,115.855307,256544.991680,0.351477,0.031238,2.190212,1.392783,36.409
1959,5637.0,170+34_btx,apo25m,2M08132471+4833394,9.315651e+17,6.0,47.709934,13.826,0.026,13.103,...,2M08132471+4833394,2.261488e+05,0.000140,123.262594,100479.567704,0.507685,0.069641,2.449678,1.551356,36.671
4092,5906.0,K2_C7_009-18_btx,apo25m,2M19203613-2913349,6.759392e+18,4.0,76.948860,12.186,0.021,11.791,...,2M19203613-2913349,1.391937e+05,0.026294,129.134726,175858.246573,0.345293,0.026479,2.179232,1.367153,37.264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379,2744.0,040-27_MGA,apo25m,2M20403505-0608501,6.908129e+18,3.0,99.571610,11.297,0.026,11.026,...,2M20403505-0608501,1.641840e+05,0.000062,264.679382,414909.018164,3.649619,0.031921,6.191144,1.396291,77.447
52,2282.0,110-62_MGA,apo25m,2M00294317-0104343,2.542664e+18,4.0,139.488220,11.157,0.027,10.918,...,2M00294317-0104343,1.722362e+05,0.022671,237.036611,461753.752616,2.657228,0.000619,5.108268,1.090280,78.657
3543,2863.0,044+37_MGA,apo25m,2M16490009+2520563,1.300325e+18,2.0,94.274030,10.886,0.021,10.546,...,2M16490009+2520563,1.339250e+05,0.090175,258.836211,807965.149023,2.416410,0.000007,4.839174,1.019449,78.933
4580,2024.0,074-35_MGA,apo25m,2M22154170+1154386,2.727666e+18,4.0,648.315600,8.162,0.020,7.953,...,2M22154170+1154386,1.349998e+07,0.732285,124.299500,516553.955149,4.305741,0.000159,6.891474,1.056170,84.674


In [11]:
printer.to_csv('APOGEE+JOKER+ESTIMATE_MASS+ERROR.csv')

In [14]:
greenobjs_rp.to_csv('APOGEE+JOKER+ESTIMATE_MASS+ERRO+RP.csv')